# HDR单帧合成(泊松融合-依赖雅可比迭代）

## 这个示例展示如何使用一个简单的算法合成HDR效果
调用接口：
-  tianmoucv.proc.reconstruct.laplacian_blending

In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
%autoreload
from IPython.display import clear_output
import sys,os,cv2,torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from tianmoucv.isp import SD2XY
from tianmoucv.proc.reconstruct import poisson_blending
from tianmoucv.data import TianmoucDataReader

## 数据读取

In [ ]:
train='/data/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]
val='/data/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]

key_list = [] #包含所有sample名作为匹配关键词
for sampleset in valdata:
    print(' ')
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本--------------------')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
for sampleset in traindata:
    print(' ')
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本--------------------')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)     
        
all_data = valdata + traindata #包含所有数据的父路径的列表

## 融合图像

In [ ]:
%autoreload
import torch.nn as nn
import math
import time

def images_to_video(frame_list,name,size=(640,320),Flip=False):
    fps = 25        
    ftmax = max([np.max(ft) for ft in frame_list])
    ftmin = min([np.min(ft) for ft in frame_list])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()

psnrcount =0
count = 0
key_list = ['test_tunnel7_hdr_ae']

for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500,matchkey=key)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    PSNR = 0
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 20:
            psnrcount += 1
            F0 = sample['F0'][0,...]
            F_HDR = sample['F0_HDR'][0,...]
            raw_F0 = F0.clone()
            #只有第0针可以合成
            for t in [0]:
                clear_output()
                tsdiff = sample['rawDiff'][0,...]/128.0        
                SD = tsdiff[1:,t,...].permute(1,2,0)
                Ix,Iy= SD2XY(SD)
                Ix = F.interpolate(torch.Tensor(Ix).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                Iy = F.interpolate(torch.Tensor(Iy).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                blend_hdr = poisson_blending(Ix,Iy, srcimg= F0,iteration=20, mask_rgb=True,mask_th=36)
                blend_hdr_more_sd = poisson_blending(Ix,Iy, srcimg= F0,iteration=20, mask_rgb=True,mask_th=48)#更激进的参数
                show = torch.cat([raw_F0,blend_hdr],dim=0)
                img_list.append(show.numpy()[...,[2,1,0]])
            
            plt.figure(figsize=(12,6))
            plt.subplot(1,3,1)  
            plt.imshow(F0)
            plt.subplot(1,3,2)  
            plt.imshow(blend_hdr)
            plt.subplot(1,3,3)  
            plt.imshow(blend_hdr_more_sd)
            plt.show()
            break
        else:
            break

In [ ]:
images_to_video(img_list,size=(640,640),name='./Direct_'+key+'.mp4')